<a href="https://colab.research.google.com/github/balezz/cv_course_fa_mag/blob/main/CodeLab_2_SoftmaxClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Линейный классификатор Softmax 

Для выполнения этого задания нужно будет дописать код в этом ноутбуке  

В этом упражнении Вам предстоит:

- реализовать функцию потерь (**loss**) для Softmax классификатора
- реализовать векторизованную функцию для вычисления **аналитического градиента**
- **оптимизировать** матрицу весов W с помощью стохастического градиентного спуска **SGD**
- найти лучшие гиперпараметры **learning rate и regularization** 
- **визуализировать** матрицу оптимальных весов.

In [ ]:
import time
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

: 

: 

# Загрузка датасета CIFAR-10 и предварительная подготовка

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Проверим размер входных и выходных векторов.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print()

In [ ]:
# Перед началом работы полезно посмотреть на данные.
# Отобразим пример из каждого класса.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Для удобства преобразуем двумерные изображения в одномерные вектора fp64
X_train = np.reshape(X_train, (X_train.shape[0], -1)).astype(np.float64)
X_test = np.reshape(X_test, (X_test.shape[0], -1)).astype(np.float64)
y_train = y_train.flatten()
y_test = y_test.flatten()
# Проверим размер полученных данных
print('Training data shape: ', X_train.shape)
print('Test data shape: ', X_test.shape)
print('Training label shape: ', y_train.shape)
print('Test label shape: ', y_test.shape)

In [ ]:
# Нормализуем значения яркости пикселей 
mean_image = np.mean(X_train, axis=0)
print(mean_image[:10]) 

# визуализируем среднюю яркость
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((32,32,3)).astype('uint8')) 
plt.show()

In [ ]:
# Вычтем средние значения яркости
X_train -= mean_image
X_test -= mean_image

# Добавим к вектору исходных данных фиктивный признак с постоянным значением 1.
# Этот трюк позволит избежать лишних вычислений: x @ W + b  => x' @ W'
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

print(X_train.shape, X_test.shape)

## Softmax Classifier


In [ ]:
# Небольшая фишка с итерацией по заданному измерению ndarray
p = np.arange(40).reshape(4, 10)
y = 1, 2, 3, 4

print('p = \n', p)
print('y = \n', y)

In [ ]:
# Возьмем первый элемент из нулевой строки, второй элемент из первой строки и так далее.
p[range(4), y]

In [ ]:
# Реализуйте эффективную векторизованную  
# функцию вычисления loss и dW

def softmax_loss(W, X, y, reg):
    """
    Функция потерь Softmax, векторизованная версия.
    Входы и выходы такие же, как у softmax_loss_naive.
    """
    # Инициализируем значения loss и градиента нулями.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Вычислить softmax loss и его градиент без явных циклов.            #
    # Сохраните значение функции потерь в переменной loss и градиент в dW.     #
    # Будьте внимательны к численной нестабильности при экспонентах. Не забывайте#
    # про регуляризацию!                                                      #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    return loss, dW


In [ ]:
# Инициализируем веса значениями близкими, но не равными нулю
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss(W, X_test, y_test, 0.0)

# Обязательно проверим правильность реализации функции
# Для 10 классов loss должен быть около -log(0.1).
print('loss: %f' % loss)
print('Начальное значение Loss = %f' % (-np.log(0.1)))

**Вопрос 1**

Объясните, почему мы предположили, что для 10 классов при весах близких к нулю
 softmax loss приблизительно равен -log(0.1)?  
$\color{blue}{\textit Ответ:}$ *заполнить здесь* 


# Стохастический градиентный спуск

In [ ]:
# Реализуйте SGD и проверьте результат 

class SoftmaxClassifier():
    def __init__(self):
        self.W = None

    def train(self, X, y, learning_rate=1e-3, reg=1e-5, num_iters=100,
              batch_size=32, verbose=True):
        """
        Обучение классификатора с помощью стохастического градиентного спуска.
        Входы:
        - X: numpy массив формы (N, D) с обучающими данными; N примеров, каждое
          размерности D.
        - y: numpy массив формы (N,) с метками; y[i] = c означает, что X[i] имеет
          метку 0 <= c < C для C классов.
        - learning_rate: (float) скорость обучения.
        - reg: (float) сила регуляризации.
        - num_iters: (integer) число шагов оптимизации.
        - batch_size: (integer) число примеров в мини-пакете.
        - verbose: (boolean) если True, выводить прогресс оптимизации.
        Возвращает:
        Список значений функции потерь на каждой итерации обучения.
        """
        num_train, dim = X.shape
        num_classes = np.max(y) + 1 # предполагаем, что y принимает значения 0...K-1
        if self.W is None:
            # лениво инициализируем W
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Запускаем стохастический градиентный спуск для оптимизации W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            #########################################################################
            # TODO:                                                                 #
            # Выберите batch_size элементов из обучающей выборки и соответствующие  #
            # метки для использования в этом шаге градиентного спуска. Сохраните    #
            # данные в X_batch и соответствующие метки в y_batch; после выборки     #
            # X_batch должен иметь размерность (batch_size, dim),                   #
            # а y_batch — (batch_size,)                                             #
            #                                                                       #
            # Подсказка: используйте np.random.choice для генерации индексов.       #
            # Выбор с возвращением (replacement) быстрее, чем без него.             #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            # вычисляем loss и градиент
            loss,  grad = self.loss(X_batch, y_batch, reg)
            loss_history.append(loss)

            # выполняем обновление параметров
            #########################################################################
            # TODO:                                                                 #
            # Обновите веса, используя градиент и скорость обучения.                #
            #########################################################################
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****



            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

            if verbose and it % 100 == 0:
                print('iteration %d / %d: loss %f '  % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        """
        Использует обученные веса этого линейного классификатора для предсказания
        меток для входных данных.
        Входы:
        - X: numpy массив формы (N, D) с данными; N примеров размерности D.
        Возвращает:
        - y_pred: массив предсказанных меток длины N, каждое значение — целое число
                  с предсказанным классом.
        """
        y_pred = np.zeros(X.shape[0])
        ###########################################################################
        # TODO:                                                                   #
        # Реализуйте этот метод. Сохраните предсказанные метки в y_pred.          #
        ###########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        return y_pred

    def loss(self, X_batch, y_batch, reg):
        """
        Вычисляет функцию потерь и её производную.
        Входы:
        - X_batch: numpy массив формы (N, D) с мини-пакетом из N точек; каждая точка размерности D.
        - y_batch: numpy массив формы (N,) с метками для мини-пакета.
        - reg: (float) сила регуляризации.
        Возвращает: кортеж из:
        - loss — одномерное число (float)
        - gradient по отношению к self.W; массив той же формы, что и W
        """
        return softmax_loss(self.W, X_batch, y_batch, reg)
    
    def save_weights(self, path):
        np.save(path, self.W)
    
    def load_weights(self, path):
        self.W = np.load(path)


In [ ]:
softmax_cls = SoftmaxClassifier()
tic = time.time()
loss_hist = softmax_cls.train(
    X_train[:1000], y_train[:1000], 
    learning_rate=1e-4, reg=1e-3,
    num_iters=1000, verbose=True)

toc = time.time()
print('That took %fs' % (toc - tic))

In [ ]:
# Построим график зависимости loss от количества итераций
plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

In [ ]:
# оцените точность предсказания на выборках train и val
y_train_pred = softmax_cls.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred) ))
y_test_pred = softmax_cls.predict(X_test)
print('validation accuracy: %f' % (np.mean(y_test == y_test_pred) ))

# Поиск лучших гиперпараметров

In [ ]:
# Используйте валидационную выборку для выбора лучших гиперпараметров 
# (learning rate and regularization strength)
# Добейтесь точности не меньше 0.38 на выборке test.
# Используйте словарь results в котором 
# ключи - кортеж  (learning_rate, regularization_strength)
# значения - (training_accuracy, validation_accuracy)
# Точность вычисляется как отношение числа верно предсказанных классов 
# к объему выборки
results = {}
best_val = -1   
best_softmax = None # Лучший экземпляр Softmax classifier 

################################################################################
# TODO:                                                                        #
# Напишите код, позволяющий найти лучшее значение гиперпараметров на val       #
# выборке. Для каждой комбинации гиперпараметров обучите классификатор         #
# на train выборке, вычислите точность на выборках train, val и сохраните      #
# результат в словарь results. Лучшее значение точности сохраните в best_val   #
# лучший классификатор - в best_softmax                                        #
#                                                                              #
################################################################################

# Пример списка допустимых значений. Можете изменить на свое усмотрение.
learning_rates = [1e-7, 5e-5]
regularization_strengths = [1e-1, 1, 10]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Вывод результатов.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Визуализируем результаты кросс-валидации
import math
import pdb

# pdb.set_trace()

x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# график accuracy на обучении
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.tight_layout(pad=3)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 training accuracy')

# график accuracy на валидации
colors = [results[x][1] for x in results] # default size of markers is 20
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors, cmap=plt.cm.coolwarm)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 validation accuracy')
plt.show()

# Точность на test выборке


In [ ]:
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

In [ ]:
best_softmax.save_weights('softmax_weights.npy')

In [ ]:
# Визуализируем веса W для каждого класса
w = best_softmax.W[:-1,:] # отбросим фиктивное измерение bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plt.figure(figsize=(16, 10))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Масштабируем веса в значения от 0 до 255 для визуализации
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'), interpolation='quadric')
    plt.axis('off')
    plt.title(classes[i])